# Здесь перечислены 3 метода улучшения F-меры

Сначала предобработаем данные и введем метрики оценки качества. 

In [163]:
import json, os
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()

pd.set_option('display.max_colwidth', 1000) 
PATH_TO_DATA = './data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [164]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp + fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp + fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
            
        if (prec + rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [167]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# 1.Используем другие стоп-слова из https://pypi.org/project/stop-words/ и выбираем не 10, а 6 самых частотных слов => F-мера улучшается до 0.18

Сначала применим только другие стоп-слова. Список этих стоп-слов шире, поэтому F-мера улучшается на 0.01 (=0.17), если мы сравниваем реальные ключевые слова с 10-ю самыми частотными словами. 

In [168]:
from string import punctuation
from stop_words import get_stop_words

punct = punctuation+'«»—…“”*№–'
stops = set(get_stop_words('ru'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
    
    return words

data['content_norm'] = data['content'].apply(normalize)

In [169]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


Теперь извлечем не топ-10, а топ-6 самых частотных слов и сравним их с реальными ключевыми словами: F-мера улучшается еще на 0.01 до 0.18. 

In [170]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(6)]))

Precision -  0.18
Recall -  0.2
F1 -  0.18
Jaccard -  0.11


# 2. Выставляем другие параметры в TfidfVectorizer: 
min_df=2 (или 3);
берется топ-5 слов вместо топ-10
===> F-мера улучшается до 0.18

In [172]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
            
stops = set(stopwords.words("russian"))
            
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
    
    return words


data['content_norm'] = data['content'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)


tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2) #меняем здесь min_df
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] #здесь -11 меняем на -6
evaluate(data['keywords'], keywords)

Precision -  0.19
Recall -  0.18
F1 -  0.18
Jaccard -  0.11


# 3. Пробуем TermExtractor ===> F-мера повышается до 0.17
https://github.com/igor-shevchenko/rutermextract

In [224]:
from rutermextract import TermExtractor
import math


term_extractor = TermExtractor()
keywords = []
for text in data['content']:
    terms = []
    for term in term_extractor(text, limit = 5):
        terms.append(term.normalized)
    keywords.append(terms)

/Users/anyway/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [ ]:
evaluate(data['keywords'], keywords)

Precision -  0.18
Recall -  0.17
F1 -  0.17
Jaccard -  0.1


In [ ]:
#не успела настроить TermExtractor так, чтобы он учитывал еще и idf-веса. 


def compute_idf(word, corpus):
#на вход берется слово, для которого считаем IDF
#и корпус документов в виде списка списков слов
#количество документов, где встречается искомый термин
#считывается как генератор списков
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

term_extractor = TermExtractor()
keywords_all = []
for text in data['content']:
    terms = []
    for term in term_extractor(text, limit = 5):
        terms.append(term.normalized)
    keywords.append(terms)
        
texts = keywords

idf = {}
for text in keywords_all:
    for word in text:
        if word not in idf:
            idf[word] = compute_idf(word, key)
        else:
            continue

print(idf)

'''keywords_ranked = []
for text in data['content_norm']:
    ms = term_extractor(''.join(text), weight=lambda term: idf.get(term), limit = 10)
    keywords_ranked.append(terms)

print(keywords_ranked) '''

/Users/anyway/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
